In [1]:
import numpy as np
import re
import itertools
from collections import Counter
from tensorflow.contrib import learn
import unittests as tests
import tensorflow as tf

# Funcoes Auxiliares (tiradas do paper)

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

def batch_iter(data, batch_size, num_batches_per_epoch, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    #pegou todos os dados de treino
    data_size = len(data)
    #meus dados foram dividos em blocos de 64 dados
    #logo, por época 150 (resultado dessa divisao) blocs são usados pra eu acessar todos os dados
    #num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            print('Epoch {:>2}, Sentence Batch {}:  '.format(epoch + 1, batch_num), end='')
            #eu tenho 150 blocos de dados que rodam 200 vezes
            #eu quero pegar só o primeiro bloco
            yield shuffled_data[start_index:end_index]

# Data explore

In [3]:
import random

data,labels = load_data_and_labels('data/rt-polaritydata/rt-polarity.pos','data/rt-polaritydata/rt-polarity.neg')

print (len(data)) #todos os reviews, cada elemento eh um review
print (len(labels)) #todos os sentimentos


10662
10662


In [4]:
(data[0], labels[0]) #tupla primeiro review, no caso eh positivo

("the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal",
 array([0, 1]))

# Pre processing
- Build vocabulary

In [5]:
max_review_size = max([len(x.split(" ")) for x in data])

#Maps documents to sequences of word ids.
#Basicamente cada palavra ja eh mapeada para um id e a sentenca tb
vocab_processor = learn.preprocessing.VocabularyProcessor(max_review_size)
x = np.array(list(vocab_processor.fit_transform(data)))#aqui que ocorre o mapeamento
(x[0], data[0])

(array([ 1,  2,  3,  4,  5,  6,  1,  7,  8,  9, 10, 11, 12, 13, 14,  9, 15,
         5, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]),
 "the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal")

In [6]:
max_review_size

56

- Randomly shuffle data

In [7]:
#Shuffle Data
np.random.seed(10) #for debugging, garante que os numeros aleatorios gerados sempre sejam os mesmos
shuffle_indices = np.random.permutation(np.arange(len(labels)))
shuffle_indices

array([ 7359,  5573, 10180, ...,  1344,  7293,  1289])

In [8]:
# basicamente eu refaço o array de arrays, mas agora cada indice eh oq ta como indice gerado do random
#ex: x[7359] == x_shuffled[0] assim os dois (data, label) mantem o mesmo indice
x_shuffled = x[shuffle_indices] #aqui sao as sentencas ja mapeadas
y_shuffled = labels[shuffle_indices]



 - Train/Validation split

In [9]:
val_percentage = .1
val_sample_index = -1 * int(val_percentage * float(len(labels)))
#TODO - fazer do meu jeito isso aqui
#justamente pra pegar os ultimos enquanto no validation, pega de ordem reversa,
#ex: pegue todos com excessao dos -1066 ultimos
#ou pegue os 1066 primeiros

In [10]:
x_train, x_val = x_shuffled[:val_sample_index], x_shuffled[val_sample_index:]
y_train, y_val = y_shuffled[:val_sample_index], y_shuffled[val_sample_index:]

In [11]:
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_val)))

Vocabulary Size: 18758
Train/Dev split: 9596/1066


# Build CNN

- Inputs and Labels instances

In [12]:
def neural_net_sentence_input(sentence_size):
    """
    Return a Tensor for a batch of image input
    : sentence_size: Size of the sentence with the biggest len
    : return: Tensor for sentences input.
    Remeber: all sentences were padded to get the max len
    """
    # TODO: Implement Function
    return tf.placeholder(tf.int32, shape=[None,sentence_size],name='input_x')


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=[None,n_classes],name='input_y')


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=None,name='keep_prob')


"""
UNIT TESTS
"""
tf.reset_default_graph()
tests.test_nn_sentence_inputs(neural_net_sentence_input)
tests.test_nn_label_inputs(neural_net_label_input)
tests.test_nn_keep_prob_inputs(neural_net_keep_prob_input)

Sentence Input Tests Passed.
Label Input Tests Passed.
Keep Prob Tests Passed.


- Load Pre Trained Word2Vec Model

In [44]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 



2017-06-11 23:50:02,527 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2017-06-11 23:51:02,271 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin


- Store only the words that exists in our vocab 

In [46]:
vocab_size = len(vocab_processor.vocabulary_)
T = np.random.rand(vocab_size, 300)
vocab_dict = vocab_processor.vocabulary_._mapping
for word,idx in vocab_dict.items():
    if word in model:
        T[idx] = model[word]
    else:
        T[idx] = np.random.uniform(low=-1, high=1, size=(300,))
del model

- Embedding Layer

In [47]:
def embedding_creation(x_tensor,vocab_size,embedding_size):
    W = tf.Variable(initial_value=tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),name="W")  
    W.assign(T)
    embedded_chars = tf.nn.embedding_lookup(W, x_tensor)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    
    return embedded_chars_expanded


tests.test_embed(embedding_creation)

Tests Passed


- Convolution Layer

In [48]:
def conv2d_maxpool(x_tensor, num_filters, filter_size):
    """
    return: A tensor that represents convolution and max pooling of x_tensor
    """
    embbeding_size = int(x_tensor.shape[2])
    filter_shape = [filter_size,embbeding_size, 1, num_filters]
    
    weights = tf.Variable(tf.truncated_normal(filter_shape,stddev=0.1), name="W")
    bias = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    """
    Strides controls how the filter convolves around the input
    As we want to go each word per time, everything will have size one
    As we apply conv layers, we could pad the image to preserve the dimension 
    (and try to extract more level features)
    Because we are only dealing with words, this would not be necessary. This is known as narrow convolution
    
    Conv gives us an output of shape [1, sequence_length - filter_size + 1, 1, 1] - There is a formula to discover that
    
    """
    conv = tf.nn.conv2d(x_tensor, weights, strides=[1, 1, 1, 1], padding='VALID')

    conv = tf.nn.bias_add(conv, bias)
    #add non linearity
    h = tf.nn.relu(conv, name="relu")
    sequence_length = int(x_tensor.shape[1])
    conv_output = [1, sequence_length - filter_size + 1, 1, 1]
    
    #Maxpooling over the outputs
    #this will heaturn a tensor of shape [batch_size, 1, 1, num_filters] 
    #which is essencialy a feature vector where the last dimension correspond to features
    #Stride have this size basically because of the same logic applied before
    pooled = tf.nn.max_pool(h, ksize=conv_output,
                            strides=[1, 1, 1, 1],
                            padding='VALID',
                            name='pool') 
    
    return pooled


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_con_pool(conv2d_maxpool)

Tests Passed


- Apply different filters

In [49]:
def apply_conv_filters(x_tensor,filter_sizes,num_filters):
# Create a convolution + maxpool layer for each filter size
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-{}".format(filter_size)):
            pooled = conv2d_maxpool(x_tensor, num_filters, filter_size)
            pooled_outputs.append(pooled)     
    num_filters_total = num_filters * len(filter_sizes)
    #concat -> sum(Daxis(i)) where Daxis is Dimension axis (in our case is the third one)
    h_pool = tf.concat(pooled_outputs, 3)
    return h_pool

tests.test_apply_filters(apply_conv_filters,conv2d_maxpool)

Tests Passed


- Flatten Layer

The flatten function to change the dimension of x_tensor from a 4-D tensor to a 2-D tensor. The output should be the shape (Batch Size, Flattened Features Size).

In [50]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    #print (tf.contrib.layers.flatten(x_tensor))
    #This is a general flatten function
    flat = x_tensor.shape[1]*x_tensor.shape[2]*x_tensor.shape[3]
    return tf.reshape(x_tensor,[-1,int(flat)])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

tests.test_flatten(flatten)

Tests Passed


In [51]:
def output(x_tensor,num_classes):
    num_filters_total = int(x_tensor.shape[1])
    W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")

    scores = tf.nn.xw_plus_b(x_tensor, W, b, name="scores")
    return scores, tf.nn.l2_loss(W), tf.nn.l2_loss(b)


- Convolutional Network

In [54]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    
    vocab_size = len(vocab_processor.vocabulary_)
    embed_dim = 300
    with tf.name_scope("embedding"):
        embbed_layer = embedding_creation(x,vocab_size,embed_dim)
    
    num_filters = 128
    filter_sizes = [3,4,5]
    conv_layer = apply_conv_filters(embbed_layer,filter_sizes,num_filters)

    

    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    flat_layer = flatten(conv_layer)
    
    with tf.name_scope("dropout"):
        dropout =  tf.nn.dropout(flat_layer, keep_prob)
        
    with tf.name_scope("output"):
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs) 
        num_classes = 2
        output_layer, l2_w, l2_b = output(dropout, num_classes)

    
    return output_layer, l2_w, l2_b

In [55]:
# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()
l2_loss = tf.constant(0.0)
l2_reg_lambda = 0.3


# Inputs
x_input = neural_net_sentence_input(56) #sequence_length
y_input = neural_net_label_input(2) #positive or negative
keep_prob = neural_net_keep_prob_input()

# Model
logits, l2_w, l2_b = conv_net(x_input, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_input))
l2_loss += l2_w
l2_loss += l2_b
cost =  cost + l2_reg_lambda * l2_loss
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y_input, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


"""
UNIT TEST
"""
tests.test_conv_net(conv_net)

# save memory
del T

Neural Network Built!


# Training Process

In [56]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    
    
    session.run(optimizer, feed_dict={
            x_input: feature_batch,
            y_input: label_batch,
            keep_prob: keep_probability})


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_train_nn(train_neural_network)

Tests Passed


# Print statistics

In [57]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss = session.run(cost, feed_dict={
            x_input: feature_batch,
            y_input: label_batch,
            keep_prob: 1.})
    
    valid_acc = session.run(accuracy, feed_dict={
        x_input: x_val,
        y_input: y_val,
        keep_prob: 1.})
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss,valid_acc))

# Hyperparameters

- Just for a Single Batch

In [58]:
epochs = 20
batch_size = 64
keep_probability =  0.5
num_batches_per_epoch = 1

# Training on a Single Batch

In [59]:
print('Checking the Training on a Single Batch...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    # Generate single batches
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, 1, epochs, shuffle=False)
    # Training cycle
    for batch in batches:
        batch_features, batch_labels = zip(*batch)
        train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
        print_stats(sess, batch_features, batch_labels, cost, accuracy)

Checking the Training on a Single Batch...
Epoch  1, Sentence Batch 0:  Loss:     1.9342 Validation Accuracy: 0.508443
Epoch  2, Sentence Batch 0:  Loss:     1.5390 Validation Accuracy: 0.506567
Epoch  3, Sentence Batch 0:  Loss:     1.1085 Validation Accuracy: 0.494371
Epoch  4, Sentence Batch 0:  Loss:     0.9760 Validation Accuracy: 0.501876
Epoch  5, Sentence Batch 0:  Loss:     0.9649 Validation Accuracy: 0.487805
Epoch  6, Sentence Batch 0:  Loss:     0.9282 Validation Accuracy: 0.495310
Epoch  7, Sentence Batch 0:  Loss:     0.8816 Validation Accuracy: 0.493433
Epoch  8, Sentence Batch 0:  Loss:     0.8661 Validation Accuracy: 0.500938
Epoch  9, Sentence Batch 0:  Loss:     0.8604 Validation Accuracy: 0.511257
Epoch 10, Sentence Batch 0:  Loss:     0.8572 Validation Accuracy: 0.515947
Epoch 11, Sentence Batch 0:  Loss:     0.8546 Validation Accuracy: 0.515009
Epoch 12, Sentence Batch 0:  Loss:     0.8520 Validation Accuracy: 0.528143
Epoch 13, Sentence Batch 0:  Loss:     0.8492

- Update Hyperparameters for full training

In [60]:
epochs = 200
num_batches_per_epoch = int((len(list(zip(x_train, y_train)))-1)/batch_size) + 1

In [ ]:
print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    # Generate single batches
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_batches_per_epoch, epochs, shuffle=True)
    # Training cycle
    for batch in batches:
        batch_features, batch_labels = zip(*batch)
        train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
        print_stats(sess, batch_features, batch_labels, cost, accuracy)

Training...
Epoch  1, Sentence Batch 0:  Loss:     1.7258 Validation Accuracy: 0.505629
Epoch  1, Sentence Batch 1:  Loss:     1.7517 Validation Accuracy: 0.492495
Epoch  1, Sentence Batch 2:  Loss:     2.2424 Validation Accuracy: 0.494371
Epoch  1, Sentence Batch 3:  Loss:     1.9205 Validation Accuracy: 0.484053
Epoch  1, Sentence Batch 4:  Loss:     2.6928 Validation Accuracy: 0.487805
Epoch  1, Sentence Batch 5:  Loss:     2.5870 Validation Accuracy: 0.488743
Epoch  1, Sentence Batch 6:  Loss:     2.2782 Validation Accuracy: 0.498124
Epoch  1, Sentence Batch 7:  Loss:     1.8721 Validation Accuracy: 0.510319
Epoch  1, Sentence Batch 8:  Loss:     2.0454 Validation Accuracy: 0.520638
Epoch  1, Sentence Batch 9:  Loss:     2.0662 Validation Accuracy: 0.518762
Epoch  1, Sentence Batch 10:  Loss:     2.3619 Validation Accuracy: 0.513133
Epoch  1, Sentence Batch 11:  Loss:     2.3981 Validation Accuracy: 0.512195
Epoch  1, Sentence Batch 12:  Loss:     2.6758 Validation Accuracy: 0.5150

Epoch  1, Sentence Batch 106:  Loss:     1.8419 Validation Accuracy: 0.548781
Epoch  1, Sentence Batch 107:  Loss:     1.5938 Validation Accuracy: 0.569418
Epoch  1, Sentence Batch 108:  Loss:     1.1906 Validation Accuracy: 0.579737
Epoch  1, Sentence Batch 109:  Loss:     1.5988 Validation Accuracy: 0.607880
Epoch  1, Sentence Batch 110:  Loss:     1.2171 Validation Accuracy: 0.611632
Epoch  1, Sentence Batch 111:  Loss:     1.5294 Validation Accuracy: 0.607880
Epoch  1, Sentence Batch 112:  Loss:     1.4758 Validation Accuracy: 0.612570
Epoch  1, Sentence Batch 113:  Loss:     1.5190 Validation Accuracy: 0.607880
Epoch  1, Sentence Batch 114:  Loss:     1.3535 Validation Accuracy: 0.608818
Epoch  1, Sentence Batch 115:  Loss:     1.2326 Validation Accuracy: 0.606942
Epoch  1, Sentence Batch 116:  Loss:     1.4366 Validation Accuracy: 0.621951
Epoch  1, Sentence Batch 117:  Loss:     1.3420 Validation Accuracy: 0.613508
Epoch  1, Sentence Batch 118:  Loss:     1.2936 Validation Accur

Epoch  2, Sentence Batch 62:  Loss:     0.8960 Validation Accuracy: 0.644465
Epoch  2, Sentence Batch 63:  Loss:     1.0110 Validation Accuracy: 0.649156
Epoch  2, Sentence Batch 64:  Loss:     0.8862 Validation Accuracy: 0.628518
Epoch  2, Sentence Batch 65:  Loss:     0.9429 Validation Accuracy: 0.620075
Epoch  2, Sentence Batch 66:  Loss:     0.9410 Validation Accuracy: 0.599437
Epoch  2, Sentence Batch 67:  Loss:     0.9278 Validation Accuracy: 0.595685
Epoch  2, Sentence Batch 68:  Loss:     0.8423 Validation Accuracy: 0.607880
Epoch  2, Sentence Batch 69:  Loss:     0.8855 Validation Accuracy: 0.627580
Epoch  2, Sentence Batch 70:  Loss:     0.9021 Validation Accuracy: 0.630394
Epoch  2, Sentence Batch 71:  Loss:     0.8462 Validation Accuracy: 0.642589
Epoch  2, Sentence Batch 72:  Loss:     0.8185 Validation Accuracy: 0.649156
Epoch  2, Sentence Batch 73:  Loss:     0.9280 Validation Accuracy: 0.651970
Epoch  2, Sentence Batch 74:  Loss:     0.8618 Validation Accuracy: 0.644465

Epoch  3, Sentence Batch 18:  Loss:     0.6904 Validation Accuracy: 0.659475
Epoch  3, Sentence Batch 19:  Loss:     0.6899 Validation Accuracy: 0.636961
Epoch  3, Sentence Batch 20:  Loss:     0.6479 Validation Accuracy: 0.641651
Epoch  3, Sentence Batch 21:  Loss:     0.6633 Validation Accuracy: 0.617261
Epoch  3, Sentence Batch 22:  Loss:     0.7518 Validation Accuracy: 0.608818
Epoch  3, Sentence Batch 23:  Loss:     0.6603 Validation Accuracy: 0.610694
Epoch  3, Sentence Batch 24:  Loss:     0.6722 Validation Accuracy: 0.610694
Epoch  3, Sentence Batch 25:  Loss:     0.6920 Validation Accuracy: 0.628518
Epoch  3, Sentence Batch 26:  Loss:     0.7327 Validation Accuracy: 0.639775
Epoch  3, Sentence Batch 27:  Loss:     0.6376 Validation Accuracy: 0.641651
Epoch  3, Sentence Batch 28:  Loss:     0.6966 Validation Accuracy: 0.653846
Epoch  3, Sentence Batch 29:  Loss:     0.6850 Validation Accuracy: 0.636023
Epoch  3, Sentence Batch 30:  Loss:     0.8048 Validation Accuracy: 0.635084

Epoch  3, Sentence Batch 124:  Loss:     0.6362 Validation Accuracy: 0.641651
Epoch  3, Sentence Batch 125:  Loss:     0.7192 Validation Accuracy: 0.648218
Epoch  3, Sentence Batch 126:  Loss:     0.6114 Validation Accuracy: 0.661351
Epoch  3, Sentence Batch 127:  Loss:     0.6889 Validation Accuracy: 0.665103
Epoch  3, Sentence Batch 128:  Loss:     0.6737 Validation Accuracy: 0.631332
Epoch  3, Sentence Batch 129:  Loss:     0.7489 Validation Accuracy: 0.606942
Epoch  3, Sentence Batch 130:  Loss:     0.7424 Validation Accuracy: 0.604128
Epoch  3, Sentence Batch 131:  Loss:     0.6701 Validation Accuracy: 0.606004
Epoch  3, Sentence Batch 132:  Loss:     0.6409 Validation Accuracy: 0.626642
Epoch  3, Sentence Batch 133:  Loss:     0.7292 Validation Accuracy: 0.646341
Epoch  3, Sentence Batch 134:  Loss:     0.6247 Validation Accuracy: 0.676360
Epoch  3, Sentence Batch 135:  Loss:     0.6253 Validation Accuracy: 0.659475
Epoch  3, Sentence Batch 136:  Loss:     0.6429 Validation Accur

...

Epoch 81, Sentence Batch 93:  Loss:     0.0047 Validation Accuracy: 0.728893
Epoch 81, Sentence Batch 94:  Loss:     0.0044 Validation Accuracy: 0.727955
Epoch 81, Sentence Batch 95:  Loss:     0.0055 Validation Accuracy: 0.725141
Epoch 81, Sentence Batch 96:  Loss:     0.0045 Validation Accuracy: 0.723265
Epoch 81, Sentence Batch 97:  Loss:     0.0045 Validation Accuracy: 0.727017
Epoch 81, Sentence Batch 98:  Loss:     0.0046 Validation Accuracy: 0.727017
Epoch 81, Sentence Batch 99:  Loss:     0.0053 Validation Accuracy: 0.725141
Epoch 81, Sentence Batch 100:  Loss:     0.0042 Validation Accuracy: 0.726079
Epoch 81, Sentence Batch 101:  Loss:     0.0050 Validation Accuracy: 0.727017
Epoch 81, Sentence Batch 102:  Loss:     0.0048 Validation Accuracy: 0.726079
Epoch 81, Sentence Batch 103:  Loss:     0.0044 Validation Accuracy: 0.727017
Epoch 81, Sentence Batch 104:  Loss:     0.0043 Validation Accuracy: 0.727017
Epoch 81, Sentence Batch 105:  Loss:     0.0041 Validation Accuracy: 0.

Epoch 82, Sentence Batch 49:  Loss:     0.0037 Validation Accuracy: 0.733584
Epoch 82, Sentence Batch 50:  Loss:     0.0039 Validation Accuracy: 0.733584
Epoch 82, Sentence Batch 51:  Loss:     0.0037 Validation Accuracy: 0.733584
Epoch 82, Sentence Batch 52:  Loss:     0.0047 Validation Accuracy: 0.733584
Epoch 82, Sentence Batch 53:  Loss:     0.0036 Validation Accuracy: 0.735460
Epoch 82, Sentence Batch 54:  Loss:     0.0036 Validation Accuracy: 0.727955
Epoch 82, Sentence Batch 55:  Loss:     0.0042 Validation Accuracy: 0.727955
Epoch 82, Sentence Batch 56:  Loss:     0.0039 Validation Accuracy: 0.724203
Epoch 82, Sentence Batch 57:  Loss:     0.0038 Validation Accuracy: 0.724203
Epoch 82, Sentence Batch 58:  Loss:     0.0037 Validation Accuracy: 0.722326
Epoch 82, Sentence Batch 59:  Loss:     0.0035 Validation Accuracy: 0.721388
Epoch 82, Sentence Batch 60:  Loss:     0.0036 Validation Accuracy: 0.723265
Epoch 82, Sentence Batch 61:  Loss:     0.0036 Validation Accuracy: 0.722326

Epoch 83, Sentence Batch 5:  Loss:     0.0034 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 6:  Loss:     0.0034 Validation Accuracy: 0.724203
Epoch 83, Sentence Batch 7:  Loss:     0.0032 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 8:  Loss:     0.0039 Validation Accuracy: 0.726079
Epoch 83, Sentence Batch 9:  Loss:     0.0039 Validation Accuracy: 0.724203
Epoch 83, Sentence Batch 10:  Loss:     0.0031 Validation Accuracy: 0.726079
Epoch 83, Sentence Batch 11:  Loss:     0.0036 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 12:  Loss:     0.0035 Validation Accuracy: 0.727955
Epoch 83, Sentence Batch 13:  Loss:     0.0032 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 14:  Loss:     0.0032 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 15:  Loss:     0.0030 Validation Accuracy: 0.726079
Epoch 83, Sentence Batch 16:  Loss:     0.0033 Validation Accuracy: 0.727017
Epoch 83, Sentence Batch 17:  Loss:     0.0042 Validation Accuracy: 0.724203
Epoc

Epoch 83, Sentence Batch 111:  Loss:     0.0035 Validation Accuracy: 0.728893
Epoch 83, Sentence Batch 112:  Loss:     0.0034 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 113:  Loss:     0.0034 Validation Accuracy: 0.730769
Epoch 83, Sentence Batch 114:  Loss:     0.0037 Validation Accuracy: 0.727955
Epoch 83, Sentence Batch 115:  Loss:     0.0038 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 116:  Loss:     0.0032 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 117:  Loss:     0.0035 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 118:  Loss:     0.0033 Validation Accuracy: 0.730769
Epoch 83, Sentence Batch 119:  Loss:     0.0034 Validation Accuracy: 0.730769
Epoch 83, Sentence Batch 120:  Loss:     0.0038 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 121:  Loss:     0.0032 Validation Accuracy: 0.728893
Epoch 83, Sentence Batch 122:  Loss:     0.0035 Validation Accuracy: 0.729831
Epoch 83, Sentence Batch 123:  Loss:     0.0032 Validation Accur

Epoch 84, Sentence Batch 67:  Loss:     0.0037 Validation Accuracy: 0.729831
Epoch 84, Sentence Batch 68:  Loss:     0.0033 Validation Accuracy: 0.729831
Epoch 84, Sentence Batch 69:  Loss:     0.0032 Validation Accuracy: 0.728893
Epoch 84, Sentence Batch 70:  Loss:     0.0030 Validation Accuracy: 0.729831
Epoch 84, Sentence Batch 71:  Loss:     0.0030 Validation Accuracy: 0.732645
Epoch 84, Sentence Batch 72:  Loss:     0.0035 Validation Accuracy: 0.731707
Epoch 84, Sentence Batch 73:  Loss:     0.0033 Validation Accuracy: 0.732645
Epoch 84, Sentence Batch 74:  Loss:     0.0031 Validation Accuracy: 0.730769
Epoch 84, Sentence Batch 75:  Loss:     0.0030 Validation Accuracy: 0.728893
Epoch 84, Sentence Batch 76:  Loss:     0.0033 Validation Accuracy: 0.728893
Epoch 84, Sentence Batch 77:  Loss:     0.0033 Validation Accuracy: 0.728893
Epoch 84, Sentence Batch 78:  Loss:     0.0032 Validation Accuracy: 0.729831
Epoch 84, Sentence Batch 79:  Loss:     0.0033 Validation Accuracy: 0.731707